This is an implementation of the paper Deep Bayesian Active Learning with Image Data using PyTorch and modAL. 

modAL is an active learning framework for Python3, designed with modularity, flexibility and extensibility in mind. Built on top of scikit-learn, it allows you to rapidly create active learning workflows with nearly complete freedom. What is more, you can easily replace parts with your custom built solutions, allowing you to design novel algorithms with ease.

Since modAL only supports sklearn models, we will also use [skorch](https://skorch.readthedocs.io/en/stable/), a scikit-learn compatible neural network library that wraps PyTorch. 

In [1]:
import torch
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from skorch import NeuralNetClassifier
from modAL.models import ActiveLearner
from batchbald_redux import batchbald
from acquisition_functions import *
import os

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
MAX_EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EXPERIMENT_COUNT = 5
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
results_path = 'results'

if not os.path.exists(results_path):
    os.makedirs(results_path)

ACQ_FUNCS = {
    "var_ratios": var_ratios,
    "mean_std": mean_std,
    "max_entropy": max_entropy,
    "bald": bald,
    "uniform": uniform,
    "batch_bald": batch_bald
}

### architecture of the network we will be using

We will use the architecture described in the paper.

In [4]:
class MLP_REG(nn.Module):
    def __init__(self,):
        super(MLP_REG, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 256),
            nn.Dropout(p=0.25),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.Dropout(p=0.25),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.Linear(128, 10),)
        
        
    def forward(self, x):
        return self.layers(x)


### read training data

In [5]:
dataset_train = MNIST('.', train=True, download=True, transform=ToTensor())
dataset_test  = MNIST('.', train=False,download=True, transform=ToTensor())

traindataloader = DataLoader(dataset_train, shuffle=True, batch_size=60000)
testdataloader  = DataLoader(dataset_test , shuffle=True, batch_size=10000)

X_train, y_train = next(iter(traindataloader))
X_test , y_test  = next(iter(testdataloader))


X_train = X_train.detach().numpy()
y_train = y_train.detach().numpy()

X_test = X_test.detach().numpy()
y_test = y_test.detach().numpy()


### preprocessing

### Active Learning Procedure

In [6]:
def active_learning_procedure(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=198,
                              n_instances=10):
    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    active_pool_size = [len(X_initial)]
    pool_size = len(X_initial)
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        pool_size = pool_size + n_instances
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
        active_pool_size.append(pool_size)
    return perf_hist, active_pool_size

In [7]:
for exp_iter in range(EXPERIMENT_COUNT):
    np.random.seed(exp_iter)
    initial_idx = np.array([],dtype=int)
    for i in range(10):
        idx = np.random.choice(np.where(y_train==i)[0], size=2, replace=False)
        initial_idx = np.concatenate((initial_idx, idx))
    
    for func_name, acquisition_func in ACQ_FUNCS.items():  
        X_initial = X_train[initial_idx]
        y_initial = y_train[initial_idx]

        X_pool = np.delete(X_train, initial_idx, axis=0)
        y_pool = np.delete(y_train, initial_idx, axis=0)

        model = MLP_REG().to(DEVICE)

        estimator = NeuralNetClassifier(model,
                                      max_epochs=MAX_EPOCHS,
                                      batch_size=BATCH_SIZE,
                                      lr=LEARNING_RATE,
                                      optimizer=torch.optim.Adam,
                                      criterion=torch.nn.CrossEntropyLoss,
                                      train_split=None,
                                      verbose=0,
                                      device=DEVICE)



        acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                          X_test,
                                                          y_test,
                                                          X_pool,
                                                          y_pool,
                                                          X_initial,
                                                          y_initial,
                                                          estimator,)
        file_name = os.path.join(results_path, func_name + "_exp_" + str(exp_iter) + ".npy")
        np.save(file_name, (acc_arr, dataset_size_arr))

Accuracy after query 1: 0.6528


KeyboardInterrupt: 